# Pandas II

## Indexação Hierárquica

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
pd.options.display.max_columns = 20
np.random.seed(42)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))

In [ ]:
data = pd.Series(np.random.uniform(size=9),
                 index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

In [ ]:
data["b"]

In [ ]:
data["b":"c"]

In [ ]:
data.loc[["b", "d"]]

In [ ]:
data.loc[:, 2] # todos os valores que possuem 2 no segundo índice

In [ ]:
data.unstack()

In [ ]:
data.unstack().stack()

In [ ]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
                     columns=[["Ohio", "Ohio", "Colorado"],
                              ["Green", "Red", "Green"]])
frame

In [ ]:
frame.index.names = ["key1", "key2"]
frame.columns.names = ["state", "color"]
frame

In [ ]:
# É possível ver quantos níveis um índice tem com o atributo nlevels
frame.index.nlevels

In [ ]:
frame.sort_index(level=1)

In [ ]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                      "c": ["one", "one", "one", "two", "two",
                            "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})
frame

In [ ]:
frame2 = frame.set_index(["c", "d"])
frame2

In [ ]:
frame2.reset_index()

## Reshape e Pivoting

In [ ]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"],
                    name="number"))
data

In [ ]:
result = data.stack()
result

In [ ]:
result.unstack()

In [ ]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64")
s2 = pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64")
data2 = pd.concat([s1, s2], keys=["one", "two"])
data2

In [ ]:
data2.unstack()

In [ ]:
data2.unstack().stack()

In [ ]:
data2.unstack().stack(dropna=False)

In [ ]:
df = pd.DataFrame({"left": result, "right": result + 5},
                  columns=pd.Index(["left", "right"], name="side"))
df

In [ ]:
df.unstack(level="state")

In [ ]:
df.unstack(level="state").stack(level="side")

In [ ]:
data = pd.read_csv("bases/macrodata.csv")
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]
data.head()

In [ ]:
periods = pd.PeriodIndex(year=data.pop("year"),
                         quarter=data.pop("quarter"),
                         name="date")
periods

In [ ]:
data.index = periods.to_timestamp("D")
data.head()

In [ ]:
data = data.reindex(columns=["realgdp", "infl", "unemp"])
data.columns.name = "item"
data.head()

In [ ]:
long_data = (data.stack()
             .reset_index()
             .rename(columns={0: "value"}))

In [ ]:
long_data[:5]

In [ ]:
pivoted = long_data.pivot(index="date", columns="item",
                          values="value")
pivoted.head()

pivot é equivalente a criar um índice hierárquico usando `set_index()`seguido da chamada do método `stack()`. 

In [ ]:
unstacked = long_data.set_index(["date", "item"]).unstack(level="item")
unstacked.head()

In [ ]:
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
                   "A": [1, 2, 3],
                   "B": [4, 5, 6],
                   "C": [7, 8, 9]})
df

In [ ]:
melted = pd.melt(df, id_vars="key")
melted
#a coluna key pode ser um indicador de grupo. Quando usamos pd.melt(), precisamos indicar
#quais colunas (caso haja) sao indicadores de grupo

In [ ]:
reshaped = melted.pivot(index="key", columns="variable",
                        values="value")
reshaped

In [ ]:
reshaped.reset_index()

In [ ]:
pd.melt(df, id_vars="key", value_vars=["A", "B"])

In [ ]:
pd.melt(df, value_vars=["A", "B", "C"])

In [ ]:
df = pd.DataFrame.from_dict({"Nome": ['Loja1', 'Loja2', 'Loja3', 'Loja4'],
                             "10/05/2023": np.random.randint(10,200, size=(1,4))[0],
                             "10/06/2023": np.random.randint(12,200, size=(1,4))[0],
                             "10/07/2023": np.random.randint(12,200, size=(1,4))[0],
                             "10/08/2023": np.random.randint(12,200, size=(1,4))[0]}, orient='columns')
df

In [ ]:
df.melt(id_vars='Nome', var_name='Data', value_name='Vendas_Total')

## Agregação de dados e operações Group

In [ ]:
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1],
                                      dtype="Int64"),
                   "data1" : np.random.standard_normal(7),
                   "data2" : np.random.standard_normal(7)})
df

In [ ]:
grouped = df["data1"].groupby(df["key1"])
grouped.mean()

In [ ]:
grouped

In [ ]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

In [ ]:
df.groupby(["key1", "key2"]).size()

In [ ]:
df.groupby("key1", dropna=False).size()
#para computar o número de valores nao-nulos, use o método count()

In [ ]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)

In [ ]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

In [ ]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces['b']

In [ ]:
df.groupby("key1")["data1"]

In [ ]:
df["data1"].groupby(df["key1"])

In [ ]:
df.groupby(["key1", "key2"])[["data2"]].mean()

In [ ]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
                      columns=["a", "b", "c", "d", "e"],
                      index=["Joe", "Steve", "Wanda", "Jill", "Trey"])
people.iloc[2:3, [1, 2]] = np.nan
people

In [ ]:
mapping = {"a": "red", "b": "red", "c": "blue",
           "d": "blue", "e": "red", "f" : "orange"}
#incluí a chave f para pontuar que grupos nao usados nao geram problemas

In [ ]:
by_column = people.groupby(mapping, axis="columns")
by_column.sum()

In [ ]:
df

In [ ]:
grouped = df.groupby("key1")
grouped["data1"].nsmallest(2)

In [ ]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

In [ ]:
tips = pd.read_csv("bases/tips.csv")
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

In [ ]:
grouped = tips.groupby(["day", "smoker"])

In [ ]:
grouped_pct = grouped["tip_pct"]
grouped_pct.agg("mean")

In [ ]:
grouped_pct.agg(["mean", "std", peak_to_peak]) 

In [ ]:
ftuples = [("Average", "mean"), ("Variance", np.var)]
grouped[["tip_pct", "total_bill"]].agg(ftuples) 

In [ ]:
grouped.agg({"tip_pct" : ["min", "max", "mean", "std"],
             "size" : "sum"})

In [ ]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]
top(tips, n=6)

In [ ]:
tips.groupby("smoker").apply(top)

In [ ]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")
#group_keys = False fará com que o índice hierárquico seja suprimido

In [ ]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})
frame.head()

In [ ]:
quartiles = pd.cut(frame["data1"], 4)
quartiles.head(8)

In [ ]:
def get_stats(group):
    return pd.DataFrame(
        {"min": group.min(), "max": group.max(),
        "count": group.count(), "mean": group.mean()}
    )

grouped = frame.groupby(quartiles)
grouped.apply(get_stats)

In [ ]:
tips.head()

In [ ]:
tips.pivot_table(index=["day", "smoker"])

In [ ]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"])

In [ ]:
tips.pivot_table(index=["time", "smoker"], columns="day",
                 values="tip_pct", aggfunc=len)

In [ ]:
tips.pivot_table(index=["time", "smoker"], columns="day",
                 values="tip_pct", aggfunc=len, fill_value=0)

In [ ]:
from io import StringIO
data = """Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep="\s+")
data.head()

In [ ]:
pd.crosstab(data["Nationality"], data["Handedness"], margins=True)
#margins = True representa a soma (nesse caso) dos elementos em cada eixo

In [ ]:
pd.crosstab([tips["time"], tips["day"]], tips["smoker"], margins=True)

## TODO Section

1. Usando o dataset Pokemon.csv, agrupe os pokemons por Type 1 e retorne uma Série ordenada pela quantidade em ordem decrescente

In [ ]:
poke = pd.read_csv('bases/Pokemon.csv')
poke.head(n=10)

In [ ]:
#resposta 1

2. Considerando o dataset tips, agrupe os dados por smoker e, para a coluna tip_pct, retorne a aplicação do método `describe()`. 

In [ ]:
#resposta

3. Considerando a Series abaixo, crie uma função que preencha os valores faltantes com a média de cada grupo específico.

In [ ]:
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]
data = pd.Series(np.random.standard_normal(8), index=states)
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

In [ ]:
#resposta